In [3]:
from clean_colored_lyrics import *
from synced_lyrics import *
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from fuzzysearch import find_near_matches
import regex
import datetime
import pandas as pd
import math
import os
from pathlib import Path

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
# with open('../json/urls.json', 'r') as file:
#     data = json.load(file)

# song_name = 'cheers to youth'
# artist = 'seventeen'

# song = next((s for s in data if s["name"] == song_name), None)
# print(song)
# album = song['album']

# raw_color_coded = get_raw_color_coded_html(song_name)
# colored_lyrics = get_colored_lyrics(raw_color_coded)

# export_testing_html(str(raw_color_coded['main_lyrics_body']))

# meta_data = get_song_info(song_name, artist, album)
# meta_data['unit'] = raw_color_coded['unit']
# synced_lyrics = meta_data['syncedLyrics']
# print(meta_data['syncedLyrics'])

In [5]:
# print(meta_data)

In [6]:
# for c in colored_lyrics:
#     print(c)
# print()

# for s in synced_lyrics:
#     print(s)
# print()
# print(len(colored_lyrics), len(synced_lyrics))

In [7]:
def look_ahead_color(c_line_og, s_line_og, debug):
    s_line = s_line_og.copy()
    s_lyric = normalize_lyrics(s_line['lyric']).strip()

    c_line = c_line_og.copy()
    c_line['delta'] = 'UNKNOWN'
    c_line['start'] = 'UNKNOWN'
    c_line['end'] = 'UNKNOWN'
    c_lyric = normalize_lyrics(c_line['lyric']).strip()


    if debug: print('s_line_before', s_line)
    if debug: print('c_line_before',c_line)

    found = False
    match_end = 0

    max_l_dist = 5
    if len(c_lyric) < 5 or len(s_lyric) < 5:
        max_l_dist = 1
    elif len(c_lyric) < 10 or len(s_lyric) < 10:
        max_l_dist = int(min(len(c_lyric), len(s_lyric)) / 3)
    
    if debug: print('max dist', max_l_dist, int(min(len(c_lyric), len(s_lyric))))
    find_match = find_near_matches(c_lyric, s_lyric, max_l_dist=max_l_dist)
    if debug: print('find_match: ', find_match)

    if find_match:
        found = find_match[0].start <= 3 and find_match[0].matched
        match_end = find_match[0].end
    else:
        prefix = os.path.commonprefix([c_lyric, s_lyric])
        if debug: print('PREFIX: ' ,prefix)
        if len(prefix) >= 4: 
            if debug: print('STARTS THE SAME')
            match_end = len(prefix)
            found = True

    if found:
        c_line['start'] = s_line['start']

        # If not exact match, update the start time for updated synced line
        if s_lyric[match_end:].strip():
            pattern = r"((\d{2}):(\d{2}):(\d{2}))"
            time = re.findall(pattern, s_line['delta'])[0]

            delta_as_int = int(time[1]) * 60 + int(time[2]) + int(time[3]) * 0.01
            ratio = float(match_end / float(len(s_lyric)))


            duration = delta_as_int * ratio
            duration = datetime.timedelta(seconds=duration)
            minute = int(duration.total_seconds()//60)
            sec = int(duration.total_seconds()%60)
            tenths = int(round((duration.total_seconds() % 1) * 100))
            c_line['delta'] = f"{minute:02}:{sec:02}:{tenths:02}"

            new_end_time = str_to_time(s_line['start'], '.') + duration
            new_end_time = f"{new_end_time.minute:02}:{new_end_time.second:02}:{new_end_time.microsecond//10000:02}"
            c_line['end'] = new_end_time
            
            duration = delta_as_int - delta_as_int * ratio
            duration = datetime.timedelta(seconds=duration)
            minute = int(duration.total_seconds()//60)
            sec = int(duration.total_seconds()%60)
            tenths = int(round((duration.total_seconds() % 1) * 100))
            s_line['delta'] = f"{minute:02}:{sec:02}:{tenths:02}"

        else:
            c_line['end'] = s_line['end']
            c_line['delta'] = getDelta(c_line, ':')
            
        c_line['lyric'] = c_line_og['lyric']

        s_line['lyric'] = s_lyric[match_end:].strip()
        s_line['start'] = c_line['end']

        if debug: print(c_line)
        if debug: print(s_line)

    return {'short_line': c_line, 'long_line': s_line, 'found': found}


In [8]:
def look_ahead_synced(s_line_og, c_line_og, debug):
    c_line = c_line_og.copy()
    c_lyric = normalize_lyrics(c_line['lyric'].lower()).strip()

    s_line = s_line_og.copy()
    s_line['member'] = 'UNKNOWN'
    s_line['section'] = 'UNKNOWN'
    s_line['line'] = 'UNKNOWN'
    s_lyric = normalize_lyrics(s_line['lyric'].lower()).strip()


    if debug: print('s_line_before', s_line)
    if debug: print('c_line_before',c_line)
    
    found = False
    match_end = 0

    max_l_dist = 5
    if len(c_lyric) < 5 or len(s_lyric) < 5:
        max_l_dist = 1
    elif len(c_lyric) < 10 or len(s_lyric) < 10:
        max_l_dist = int(min(len(c_lyric), len(s_lyric)) / 3)
    
    if debug: print('max dist', max_l_dist, int(min(len(c_lyric), len(s_lyric))))
    find_match = find_near_matches(s_lyric, c_lyric, max_l_dist=max_l_dist)
    if debug: print('find_match: ', find_match)

    if find_match:
        found = find_match[0].start <= 3 and find_match[0].matched
        match_end = find_match[0].end
    else:
        prefix = os.path.commonprefix([c_lyric, s_lyric])
        if debug: print('PREFIX: ' ,prefix)
        if len(prefix) >= 4: 
            if debug: print('STARTS THE SAME')
            match_end = len(prefix)
            found = True

    if found:
        pattern = r"((\d{2}):(\d{2}):(\d{2}))"
        time = re.findall(pattern, s_line['delta'])[0]

        delta_as_int = int(time[1]) * 60 + int(time[2]) + int(time[3]) * 0.01
        ratio = float(match_end) / float(len(c_lyric))

        duration = delta_as_int * ratio
        duration = datetime.timedelta(seconds=duration)
        minute = int(duration.total_seconds()//60)
        sec = int(duration.total_seconds()%60)
        tenths = int(round((duration.total_seconds() % 1) * 100))
        s_line['delta'] = f"{minute:02}:{sec:02}:{tenths:02}"

        new_end_time = str_to_time(s_line['start'], '.') + duration
        new_end_time = f"{new_end_time.minute:02}:{new_end_time.second:02}:{new_end_time.microsecond//10000:02}"
        # s_line['end'] = new_end_time
        s_line['member'] = c_line['member']
        s_line['section'] = c_line['section']
        s_line['line'] = c_line['line']
        
        c_line['lyric'] = c_lyric[match_end:].strip()
        if debug: print('s_line', s_line)
        if debug: print('c_line',c_line)

        
    return {'short_line': s_line, 'long_line': c_line, 'found': found}


In [9]:
def merge_singular(colored_lyrics, synced_lyrics, looking_ahead_max, debug):    
    merged_lines = []

    unmerged_line_color = []
    unmerged_line_synced = []

    c_i = 0
    s_i = 0

    # colored_lyrics_copy = colored_lyrics[78:100].copy()
    # synced_lyrics_copy = synced_lyrics[50:70].copy()

    colored_lyrics_copy = colored_lyrics.copy()
    synced_lyrics_copy = synced_lyrics.copy()

    looking_ahead_index = 0
    looking_ahead = []

    find_children = False
    parent = True

    default_merge_choice_is_color = False

    if len(colored_lyrics) > len(synced_lyrics):
        default_merge_choice_is_color = True
    else:
        default_merge_choice_is_color = False

    if debug: print(default_merge_choice_is_color, len(colored_lyrics_copy), len(synced_lyrics_copy))

    while c_i < len(colored_lyrics_copy) or s_i < len(synced_lyrics_copy):
        
        if debug: print('c_i',c_i, '  s_i', s_i, '  looking_ahead_index', looking_ahead_index)

        if c_i >= len(colored_lyrics_copy):
            if debug: print('MERGE REMAINING SYNCED')
            for s in synced_lyrics_copy[s_i:]:
                unmerged_line_synced.append(s)
                if debug: print('ADDING UMERGED SYNCED --->', s)
            break

        if s_i >= len(synced_lyrics_copy):
            if debug: print('MERGE REMAINING COLOR')
            for c in colored_lyrics_copy[c_i:]:
                unmerged_line_synced.append(c)
                if debug: print('ADDING UMERGED COLOR --->', c)
            break
        
        if c_i + looking_ahead_max >= len(colored_lyrics_copy):
            if debug: print("OVER")
            looking_ahead_max = max(0, len(colored_lyrics_copy) - c_i - 1)
            if debug: print(looking_ahead_max, c_i, s_i)
            # break
        elif s_i + looking_ahead_max >= len(synced_lyrics_copy):
            if debug: print("OVER")
            looking_ahead_max = max(0, len(synced_lyrics_copy) - s_i - 1)
            if debug: print(looking_ahead_max, c_i, s_i)
            # break
        
        c_line = colored_lyrics_copy[c_i + looking_ahead_index]
        s_line = synced_lyrics_copy[s_i]

        c_lyric = c_line['lyric']
        c_lyric_norm = normalize_lyrics(c_lyric)
        s_lyric = s_line['lyric']
        s_lyric_norm = normalize_lyrics(s_lyric)
        if debug: print(c_lyric_norm, s_lyric_norm)

        if c_lyric_norm == s_lyric_norm:
            if debug: print(c_lyric, s_lyric)
            new_line = {'start': s_line['start'], 'end': s_line['end'], 'lyric': c_lyric, 'member': c_line['member'], 'delta': s_line['delta'], 'section': c_line['section'], 'line': c_line['line']}
            merged_lines.append(new_line)
            
            if debug: print("EXACT MATCH: ", new_line)
            if debug: print()
            c_i += (looking_ahead_index + 1)
            s_i += 1
            looking_ahead_index = 0
        else:
            color_is_short = True
            if debug: print("LENGETH: ",len(c_lyric_norm),len(s_lyric_norm))
            if len(c_lyric_norm) <= len(s_lyric_norm):
                if debug: print('color shorter')
                looking_ahead = look_ahead_color(c_line, s_line, debug)
            else:
                color_is_short = False
                if debug: print('synced shorter')
                looking_ahead = look_ahead_synced(s_line, c_line, debug)

            if looking_ahead_index:
                parent = color_is_short

            if debug: print('LOOK AROUND --->', looking_ahead)
            if looking_ahead['found']:
                merged_lines.append(looking_ahead['short_line'])
                if debug: print('MERGE --->', looking_ahead['short_line'])

                if (looking_ahead_index <= looking_ahead_max) :
                    if debug: print("inc colored inside")
                    for l in range(looking_ahead_index):
                        unmerged_line_color.append(colored_lyrics_copy[c_i+l])
                        if debug: print('ADDING UMERGED COLOR: ', l, ' --->', colored_lyrics_copy[c_i+l])
                            
                if color_is_short:
                    if not looking_ahead['long_line']['lyric']:
                        s_i += 1
                        c_i += (looking_ahead_index + 1)
                        if debug: print("EXACT MATCH")
                        find_children = False
                                
                    else:
                        old = synced_lyrics_copy[s_i]
                        synced_lyrics_copy[s_i] = looking_ahead['long_line']
                        if debug: print("PARTIAL MATCH ---> replacing in synced ", old, synced_lyrics_copy[s_i], ' at ', s_i)

                        c_i += (1 + looking_ahead_index)
                        find_children = True
                else:
                    if not looking_ahead['long_line']['lyric']:
                        c_i += (1 + looking_ahead_index)
                        s_i += 1
                        if debug: print("EXACT MATCH")
                        find_children = False
                    else:
                        old = colored_lyrics_copy[c_i + looking_ahead_index]
                        colored_lyrics_copy[c_i + looking_ahead_index] = looking_ahead['long_line']
                        if debug: print("PARTIAL MATCH ---> replacing in colored ", old, colored_lyrics_copy[c_i + looking_ahead_index], ' at ', c_i + looking_ahead_index)
                        s_i += 1
                        c_i += looking_ahead_index
                        find_children = True

                looking_ahead_index = 0
            else:
                if debug: print('not found', looking_ahead_index)
                looking_ahead_index += 1

        if looking_ahead_index > looking_ahead_max:
            if find_children:
                if parent:
                    if debug: print("inc colored")
                    c_i += 1
                    unmerged_line_color.append(colored_lyrics_copy[c_i])
                    if debug: print('ADDING UMERGED COLOR --->', colored_lyrics_copy[c_i])
                else:
                    if debug: print("inc short")
                    unmerged_line_synced.append(synced_lyrics_copy[s_i])
                    if debug: print('ADDING UMERGED SYNCED --->', synced_lyrics_copy[s_i])
                    s_i += 1
            else:
                if debug: print('ADDING UMERGED COLOR --->', colored_lyrics_copy[c_i])
                if debug: print('ADDING UMERGED SYNCED --->', synced_lyrics_copy[s_i])

                unmerged_line_color.append(colored_lyrics_copy[c_i])
                unmerged_line_synced.append(synced_lyrics_copy[s_i])
                c_i += 1
                s_i += 1
            find_children = False
            if debug: print('rest looking_ahead_index')
            
            looking_ahead_index = 0
        
        if debug: print()

    key_order = ['start', 'end', 'delta', 'section', 'line', 'member', 'lyric']

    merged_lines = [
        {k: entry[k] for k in key_order if k in entry} 
        for entry in merged_lines
    ]
    unmerged_line_color = [
        {k: entry[k] for k in key_order if k in entry} 
        for entry in unmerged_line_color
    ]
    unmerged_line_synced = [
        {k: entry[k] for k in key_order if k in entry} 
        for entry in unmerged_line_synced
    ]

        
    return {"merged_lines": merged_lines, "unmerged_line_color": unmerged_line_color, "unmerged_line_synced": unmerged_line_synced}

# merge_singular(colored_lyrics, synced_lyrics, True)

In [10]:
def main():
    with open('../json/urls.json', 'r') as file:
        data = json.load(file)

    print(data)

    overall_data = []
    debug = False

    for d in data:
        song_name = d['name']
        artist = 'seventeen'
        album = d['album']

        print(song_name)
        raw_color_coded = get_raw_color_coded_html(song_name)
        colored_lyrics = get_colored_lyrics(raw_color_coded)
        export_testing_html(str(raw_color_coded['main_lyrics_body']))
        meta_data = get_song_info(song_name, artist, album)

        if meta_data == 'song not found':
            print('song not found')
            continue
        
        meta_data['unit'] = raw_color_coded['unit']
        synced_lyrics = meta_data['syncedLyrics']

        looking_ahead_max = 3
        merged = merge_singular(colored_lyrics, synced_lyrics, looking_ahead_max, debug)
        meta_data['syncedLyrics'] = merged['merged_lines']
        meta_data['unmerged_line_color'] = merged['unmerged_line_color']
        meta_data['unmerged_line_synced'] = merged['unmerged_line_synced']
        
        song_name = song_name.replace(' ', '_')
        with open('../json/songs/'+ song_name +'.json', 'w') as file:
                json.dump(meta_data, file, indent=4)

        print()
    
    print('done')

    #     overall_data.append(meta_data)

    # with open('../json/meta_data.json', 'w') as file:
    #         json.dump(overall_data, file, indent=4)
main()

[{'name': 'rock with you', 'url': 'https://colorcodedlyrics.com/2021/10/21/seventeen-rock-with-you/', 'album': "SEVENTEEN 9th Mini Album 'Attacca' - SEVENTEEN"}, {'name': 'very nice', 'url': 'https://colorcodedlyrics.com/2016/07/03/seventeen-very-nice-aju-nice/', 'album': "SEVENTEEN 1st Mini Album '17 CARAT' - SEVENTEEN"}, {'name': 'shining diamond', 'url': 'https://colorcodedlyrics.com/2015/06/01/seventeen-sebeuntin-shining-diamond/', 'album': "SEVENTEEN 1st Mini Album '17 CARAT' - SEVENTEEN"}, {'name': 'super', 'url': 'https://colorcodedlyrics.com/2023/04/24/seventeen-super-sonogong/', 'album': "SEVENTEEN 10th Mini Album 'FML' - EP - SEVENTEEN"}, {'name': 'hot', 'url': 'https://colorcodedlyrics.com/2022/05/27/seventeen-hot/', 'album': "SEVENTEEN 4th Album 'Face the Sun' - SEVENTEEN"}, {'name': 'cheers', 'url': 'https://colorcodedlyrics.com/2022/07/18/seventeen-leaders-cheers/', 'album': "SEVENTEEN 4th Album Repackage 'SECTOR 17' - SEVENTEEN"}, {'name': 'to you', 'url': 'https://color

In [20]:
def singular():        
        with open('../json/urls.json', 'r') as file:
                data = json.load(file)

        overall_data = []
        debug = True

        song_name = input()
        artist = 'seventeen'

        song = next((s for s in data if s["name"] == song_name), None)
        print(song)
        album = song['album']

        raw_color_coded = get_raw_color_coded_html(song_name)
        colored_lyrics = get_colored_lyrics(raw_color_coded)
        export_testing_html(str(raw_color_coded['main_lyrics_body']))
        meta_data = get_song_info(song_name, artist, album)


        meta_data['unit'] = raw_color_coded['unit']
        synced_lyrics = meta_data['syncedLyrics']

        looking_ahead_max = 1
        merged = merge_singular(colored_lyrics, synced_lyrics, looking_ahead_max, debug)
        meta_data['syncedLyrics'] = merged['merged_lines']
        meta_data['unmerged_line_color'] = merged['unmerged_line_color']
        meta_data['unmerged_line_synced'] = merged['unmerged_line_synced']

        song_name = song_name.replace(' ', '_')
        with open('../json/songs/'+ song_name +'.json', 'w') as file:
                json.dump(meta_data, file, indent=4)
singular()

{'name': 'shining diamond', 'url': 'https://colorcodedlyrics.com/2015/06/01/seventeen-sebeuntin-shining-diamond/', 'album': "SEVENTEEN 1st Mini Album '17 CARAT' - SEVENTEEN"}
url found: https://colorcodedlyrics.com/2015/06/01/seventeen-sebeuntin-shining-diamond/
found raw color html
requesting  https://lrclib.net/api/get?artist_name=seventeen&track_name=shining_diamond&albumName=SEVENTEEN_1st_Mini_Album_'17_CARAT'_-_SEVENTEEN
file not found ../official_times/shining diamond.srt
True 101 67
c_i 0   s_i 0   looking_ahead_index 0
slipintothediamondlifeoh slipintothediamondlifeoh
Slip into the diamond life oh! Slip into the diamond life, oh
EXACT MATCH:  {'start': '00:00:29', 'end': '00:02:57', 'lyric': 'Slip into the diamond life oh!', 'member': ['Jun'], 'delta': '00:02:28', 'section': 0, 'line': 0}


c_i 1   s_i 1   looking_ahead_index 0
slipintothediamondlifeoh slipintothediamondlifeoh
 Slip into the diamond life oh! Slip into the diamond life, oh
EXACT MATCH:  {'start': '00:02:57', 'en

In [12]:
# print(raw_color_coded)

In [21]:
directory_path = Path('../json/songs')
files = [p for p in directory_path.iterdir() if p.is_file()]

meta_data = []
for file in files:
    print(file)

    with open(file, 'r') as f:
        data = json.load(f)

    meta_data.append(data)

# with open('../json/songs/lalali.json', 'r') as f:
#     meta_data = json.load(f)

with open('../json/meta_data.json', 'w') as file:
        json.dump(meta_data, file, indent=4)

../json/songs/lalali.json
../json/songs/very_nice.json
../json/songs/cheers_to_youth.json
../json/songs/bad_influence.json
../json/songs/god_of_music.json
../json/songs/thanks.json
../json/songs/rock_with_you.json
../json/songs/hot.json
../json/songs/love_money_fame.json
../json/songs/to_you.json
../json/songs/spell.json
../json/songs/f*ck_my_life.json
../json/songs/darl+ing.json
../json/songs/left_&_right.json
../json/songs/world.json
../json/songs/cheers.json
../json/songs/highlight.json
../json/songs/shining_diamond.json
../json/songs/super.json
